In [ ]:
import os

In [ ]:
os.chdir("/content/fsl")

In [ ]:
# import packages
from tqdm import tqdm

import torch as th
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
from torch.nn.modules.loss import CrossEntropyLoss, MSELoss

from torchvision.transforms import Compose, Grayscale, CenterCrop, ToTensor, ToPILImage, Resize
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.datasets import ImageFolder

from src import Learner, TaskSampler, PrototypicalNetworks, RelationNetworks, CNNEncoder

In [ ]:
# configurations
CONFIG = {}
CONFIG['shot'] = 5
CONFIG['way'] = 5
CONFIG['query'] = 10
DEVICE = 'cuda' if th.cuda.is_available() else 'cpu'
if DEVICE is 'cuda':
  print ('cuda is available as device')

cuda is available as device


In [ ]:
# Image transformation pipeline

image_transform_pipe = Compose(
    [
        Grayscale(num_output_channels=3),
        CenterCrop(224),
        Resize(128),
        ToTensor()
    ]
)

# load Train dataset
train_dataset = ImageFolder(root = "./few_shot_data/train", transform=image_transform_pipe)
# load Validation dataset
val_dataset = ImageFolder(root = "./few_shot_data/val", transform=image_transform_pipe)
# load Test dataset
test_dataset = ImageFolder(root ="./few_shot_data/test", transform=image_transform_pipe)

In [ ]:
# Create task samplers
train_sampler = TaskSampler(train_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=500)

validation_sampler = TaskSampler(val_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=100)

test_sampler = TaskSampler(test_dataset, n_way=CONFIG['way'], n_shot=CONFIG['shot'], n_query=CONFIG['query'], n_tasks=20)

In [ ]:
# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_sampler=train_sampler,
    num_workers=2,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_sampler=validation_sampler,
    num_workers=2,
    pin_memory=True,
    collate_fn=validation_sampler.episodic_collate_fn,
)

test_loader = DataLoader(
    test_dataset,
    batch_sampler=test_sampler,
    num_workers=2,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

## Run a baseline model.
Use an unfitted Prototypical Networks model, with an ImageNet pretrained Resnet model as backbone. Result gives a baseline result for a just out of the box model

In [ ]:
# load pretrained model for backbone
pretrained_weights = ResNet18_Weights.IMAGENET1K_V1
backbone_model = resnet18(weights=pretrained_weights)

# Change final layer output from number of classes 
# to flattened feature vector from resnet's preceding architecure
backbone_model.fc = nn.Flatten()
model = backbone_model.cuda()

In [ ]:
# Create baseline FSL classifier
baseline_classifier = PrototypicalNetworks(backbone=model, output_softmax_score=True)

In [ ]:
# Run baseline test prediction
baseline_classifier.eval()
with th.no_grad():
  with tqdm(test_loader, total=20) as prediction_tasks:
    total_predictions = 0
    total_correct_predictions = 0
    for task_support_images,\
      task_support_labels, \
      task_query_images, \
      task_query_labels, \
      task_class_ids in prediction_tasks:

      task_prediction_scores = baseline_classifier(
          task_support_images.to(DEVICE), 
          task_support_labels.to(DEVICE), 
          task_query_images.to(DEVICE)
        )
      
      task_prediction_labels = th.argmax(task_prediction_scores,-1)
      task_correct_predictions = (task_prediction_labels == task_query_labels.to(DEVICE)).float()
      task_accuracy = th.mean(task_correct_predictions).item()

      total_predictions += len(task_prediction_labels)
      total_correct_predictions += th.sum(task_correct_predictions).item()

      prediction_tasks.set_postfix(task_accuracy = task_accuracy)
    
    overall_accuracy = total_correct_predictions / total_predictions
  
  print (f"\n Test overall average accuracy: {overall_accuracy}")

100%|██████████| 20/20 [00:04<00:00,  4.72it/s, task_accuracy=0.6]


 Test overall average accuracy: 0.582


## Understand basic performance of Prototypical Networks after simple training

In [ ]:
# load pretrained model for backbone
pretrained_weights = ResNet18_Weights.IMAGENET1K_V1
backbone_model_1 = resnet18(weights=pretrained_weights)

# Change final layer output from number of classes 
# to flattened feature vector from resnet's preceding architecure
backbone_model_1.fc = nn.Flatten()
model_1 = backbone_model_1.cuda()

classifier_1 = PrototypicalNetworks(backbone=model_1, output_softmax_score=False)

In [ ]:
learning_rate = 0.05
optimizer_1 = SGD(classifier_1.parameters(), lr=learning_rate)
loss_function = CrossEntropyLoss()

c1_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = classifier_1,
    optimizer=optimizer_1, 
    loss_function = loss_function,
    epochs = 10,
    tensorboard_log_path = None
  )

Training Epoch 0


100%|██████████| 500/500 [00:26<00:00, 19.23it/s, episode_accuracy=0.94, episode_loss=0.185, epoch_accuracy=0.892, epoch_loss=0.308]


Validating Epoch 0


100%|██████████| 100/100 [00:05<00:00, 18.42it/s, episode_accuracy=1, overall_accuracy=0.874]


Training Epoch 1


...

100%|██████████| 100/100 [00:05<00:00, 17.68it/s, episode_accuracy=0.92, overall_accuracy=0.877]


Training Epoch 9


100%|██████████| 500/500 [00:26<00:00, 19.05it/s, episode_accuracy=0.98, episode_loss=0.0528, epoch_accuracy=0.976, epoch_loss=0.0715]


Validating Epoch 9


100%|██████████| 100/100 [00:05<00:00, 17.56it/s, episode_accuracy=0.94, overall_accuracy=0.91]


In [ ]:
c1_best_val_accuracy

0.9138

## Understand basic performance of Relation Networks after simple training

In [ ]:
# Load pretrained CNNEncoder, trained on miniImageNet on paper
pretrained_weights = th.load("./pretrained_model/miniimagenet_feature_encoder_5way_5shot.pkl", map_location=th.device('cpu'))
pretrained_encoder = CNNEncoder()
pretrained_encoder.load_state_dict(pretrained_weights)

model_2 = pretrained_encoder.cuda()
classifier_2 = RelationNetworks(backbone=model_2, output_softmax_score=False)

In [ ]:
learning_rate = 0.05
optimizer_2 = SGD(classifier_2.parameters(), lr=learning_rate)
loss_function = MSELoss()

c2_best_val_accuracy = Learner.fit(
    train_data_loader = train_loader,
    val_data_loader = val_loader,
    model = classifier_2,
    optimizer=optimizer_2, 
    loss_function = loss_function,
    epochs = 10,
    tensorboard_log_path = None
  )

Training Epoch 0


100%|██████████| 500/500 [01:34<00:00,  5.28it/s, episode_accuracy=0.28, episode_loss=0.158, epoch_accuracy=0.231, epoch_loss=0.161]


Validating Epoch 0


100%|██████████| 100/100 [00:16<00:00,  5.89it/s, episode_accuracy=0.28, overall_accuracy=0.263]


Training Epoch 1


...

100%|██████████| 100/100 [00:15<00:00,  6.58it/s, episode_accuracy=0.5, overall_accuracy=0.544]


Training Epoch 9


100%|██████████| 500/500 [01:20<00:00,  6.18it/s, episode_accuracy=0.62, episode_loss=0.107, epoch_accuracy=0.602, epoch_loss=0.116]


Validating Epoch 9


100%|██████████| 100/100 [00:15<00:00,  6.61it/s, episode_accuracy=0.44, overall_accuracy=0.56]


In [ ]:
c2_best_val_accuracy

0.56